In [6]:
from dotenv import load_dotenv
load_dotenv()
import os
import openai
import pinecone
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [4]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag" , dimension = 1536, metric="cosine",spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [5]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'Professor': 'Dr. John Smith',
  'subject': 'Physics',
  'stars': 4,
  'review': 'Dr. Smith explains concepts well but sometimes rushes through material.'},
 {'Professor': 'Dr. Emily Johnson',
  'subject': 'Mathematics',
  'stars': 5,
  'review': 'She is an excellent professor. Her lectures are clear and very organized.'},
 {'Professor': 'Dr. Michael Brown',
  'subject': 'Chemistry',
  'stars': 3,
  'review': "His classes are tough, but you'll learn a lot if you keep up with the material."},
 {'Professor': 'Dr. Sarah Davis',
  'subject': 'Biology',
  'stars': 2,
  'review': 'Dr. Davis is knowledgeable but often unapproachable and hard to understand.'},
 {'Professor': 'Dr. William Martinez',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'His passion for the subject is contagious, and he really cares about his students.'},
 {'Professor': 'Dr. Patricia Garcia',
  'subject': 'English',
  'stars': 4,
  'review': 'She is a great professor, though the workload can be overwhelmi

In [9]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input = review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["Professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [10]:
processed_data[0]

{'values': [-0.009886906,
  0.045375098,
  -0.0065170126,
  -0.013253263,
  0.008585625,
  0.020212287,
  0.023324046,
  0.01564366,
  0.035954956,
  -4.1272793e-05,
  0.0020084989,
  -0.037369393,
  -0.011633734,
  0.007630881,
  0.049844716,
  -0.04529023,
  -0.0154032055,
  -0.033776727,
  -0.0020862927,
  0.028614035,
  0.019208038,
  0.0038472652,
  0.011124537,
  0.016789353,
  -0.060820736,
  -0.009356492,
  0.056690585,
  0.04048115,
  0.03657731,
  0.013507862,
  0.09171201,
  -0.02797754,
  0.03567207,
  -0.022164209,
  -0.038218055,
  0.053352516,
  0.008267377,
  0.020495174,
  0.008811935,
  -0.0023072984,
  0.011923693,
  0.033861592,
  -0.06975997,
  -0.031089298,
  0.07355066,
  -6.464413e-05,
  0.003283259,
  -0.033748437,
  0.008826079,
  0.056888606,
  -0.030410368,
  0.032022826,
  0.009971772,
  0.01100431,
  -0.018811995,
  0.034059614,
  0.028741334,
  -0.0013251494,
  0.018104777,
  -0.024285862,
  0.087298974,
  -0.026449949,
  -0.025106234,
  -0.011386208,
  -

In [11]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [12]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}